# Graph databases

## Jakub Wilkowski

# Graphs

 * ordered pair G = (V, E) comprising a set V of vertices (nodes, points) together with a set E of edges (arcs, lines), which are 2-element subsets of V
 * Thanks Euler, thanks...
 
<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/bridges.png?raw=true"/>

[source](https://en.wikipedia.org/wiki/Seven_Bridges_of_K%C3%B6nigsberg)

# Graph algorithms

 * Shortest path
 * Minimum spanning tree
 * Strongly connected components
 * and many, many more
 

# Graph databases

 * There are no isolated pieces of information, but rich, connected domains all around us.
 * a graph database stores connections as first class citizens
 

# Top players

<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/graphdb_popularity.png?raw=true">

[source](https://db-engines.com/en/ranking/graph+dbms)

# Wait, but what's wrong with traditional relational databases?

* RDBMS are great for storing data that is consistent, well structured
* RDBMS do not like recursive queries
* RDBMS are not that good for discovering patterns
* RDBMS have a lot of overhead with many-to-many relations

# Relationships are important in life

<iframe src="https://giphy.com/embed/R8RscTZV7Iy7m" width="480" height="362" frameBorder="0" class="giphy-embed" allowFullScreen></iframe>


# RDBMS way

<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/m2m_rdbms.png?raw=true">

[source](https://neo4j.com/developer/graph-db-vs-rdbms/#_relational_databases)

# And graph database way

<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/m2m_graph.png?raw=true">

[source](https://neo4j.com/developer/graph-db-vs-rdbms/#_relational_databases)

# Property graph data model

<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/property_graph.png?raw=true">

# Query language(s):

 * **Cypher**
 * Gremlin
 * SPARQL


# Typical use cases

1. Fraud detection
1. Recommendation engines



# Data modeling (RDBMS -> Graph)

 1. Identify the nodes
 2. Assign labels to nodes (zero, one or more)
 3. Find relations between nodes
 4. Assign properties to nodes and relations
 
 
 > Two _people_, **John** and **Sally**, \_are friends\_. Both John and Sally \_have read\_ the _book_, **Graph Databases**.
 

 [source](https://neo4j.com/developer/guide-data-modeling/)

# Neo4j

 * ACID compliant
 * written in Java
 * v1 in 2010 (now 3.3.3)
 * Community/Enterprise/Government
 * own bolt protocol on port 7474

# Who uses Neo4j?

 * Walmart
 * Airbnb
 * Microsoft
 * IBM
 * ebay
 * NASA
 * Orange
 * CISCO
 * LinkedIn China
 * TomTom
 
 
 [source](https://neo4j.com/customers/)

# Cypher

 * declarative graph query language
 * created by Neo Technology
 * opened up in October 2015
 * ASCII like

```
( )                    <- node
--                     <- relation
-->                    <- directed relation
-[:LABEL]-             <- labeled relation
()-->()                <- two nodes and one direction

```

<img src="https://github.com/jakubwilkowski/lunch_GraphDB/blob/master/img/cypher.png?raw=true">

[source](https://neo4j.com/developer/cypher-query-language/)

## Query examples
`:play movie-graph`

# Examples: Cypher vs SQL

## Q1: List all Tom Hanks movies...

### Cypher: 
```SQL
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies) 
RETURN tom,tomHanksMovies
```

### SQL
```SQL
select p.name, m.title
from person p
inner join movieperson mp
  on mp.person_id = p.id
inner join movie m
  on m.id = mp.movie_id
where p.name = 'Tom Hanks' and mp.relation_type = 'acted_in';

```


## Q2: Tom Hanks' co-actors...
### Cypher: 
```SQL
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) 
RETURN distinct coActors.name
```

### SQL
```SQL
with tom_hanks_movies as (
select m.id
from person p
inner join movieperson mp
  on mp.person_id = p.id
inner join movie m
  on m.id = mp.movie_id
where p.name = 'Tom Hanks' and mp.relation_type = 'acted_in'
)

select distinct p.name
from person p
inner join movieperson mp
  on mp.person_id = p.id
inner join movie m
  on m.id = mp.movie_id
where mp.movie_id in (select id from tom_hanks_movies) 
  and mp.relation_type = 'acted_in'
  and p.name != 'Tom Hanks';

```

## Q3: How people are related to "Cloud Atlas"...
### Cypher: 
```SQL
MATCH (people:Person)-[relatedTo]-(:Movie {title: "Cloud Atlas"}) 
RETURN people.name, Type(relatedTo), relatedTo
```

### SQL

```SQL
select p.name, mp.relation_type
from person p
inner join movieperson mp
  on mp.person_id = p.id
inner join movie m
  on m.id = mp.movie_id
where m.title = 'Cloud Atlas';
```

### Note:
I chose a naive schema, what if I decided to create separate table for each relation, ie. moviedirector, movieactor, movieproducer, etc?
-> SELECT ... FROM moviedirector UNION SELECT ... FROM movieactor UNION SELECT ... FROM movieproducer;

## Q4: Movies and actors up to 4 "hops" away from Kevin Bacon
### Cypher: 
```SQL
MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood)
RETURN DISTINCT hollywood
```

### SQL
```SQL
with recursive bacon(pid, mid, lvl) as (
select distinct p.id, m.id, 1
from person p
inner join movieperson mp
  on mp.person_id = p.id
inner join movie m
  on m.id = mp.movie_id
where p.name = 'Kevin Bacon'
UNION ALL
select distinct mp.person_id, mp.movie_id, b.lvl + 1
from movieperson mp
inner join bacon b
  on (b.mid = mp.movie_id OR mp.person_id = b.pid)
where (b.lvl < 4)
)
```

## Q4: continued

```SQL
select distinct p.name
from bacon b
inner join person p
  on p.id = b.pid
UNION ALL
select distinct m.title
from bacon b
inner join movie m
  on m.id = b.mid;
```

## Q5: Bacon path, the shortest path of any relationships to Meg Ryan
### Cypher: 
```SQL
MATCH (bacon:Person {name:"Kevin Bacon"})-[*]-(meg:Person {name:"Meg Ryan"})
RETURN bacon, meg;
```

### SQL
```SQL
with recursive bacon(pid, mid, lvl) as (
select distinct p.id, m.id, 1
from person p
inner join movieperson mp
	on mp.person_id = p.id
inner join movie m
	on m.id = mp.movie_id
where p.name = 'Kevin Bacon' and mp.relation_type = 'acted_in'
UNION ALL
select distinct mp.person_id, mp.movie_id, b.lvl + 1
from movieperson mp
inner join bacon b
	on (b.mid = mp.movie_id OR mp.person_id = b.pid)
where mp.relation_type = 'acted_in' and (b.lvl < 10)
)
```

## Q5: continued

```SQL
select min(b.lvl)
from bacon b
inner join person p
	on p.id = b.pid
where  p.name = 'Meg Ryan';
```

# Neo4j & python

# py2neo

[docs](http://py2neo.org/2.0/)

## Quick setup

```
docker run --publish=7474:7474 \
    --publish=7687:7687 \
    --volume=$HOME/neo4j/data:/data \
    --env=NEO4J_AUTH=none \
    neo4j
```

In [2]:
from py2neo import Graph, Relationship, NodeSelector

graph = Graph("http://localhost:7474/")

## Some examples

## Select Tom Hanks


In [9]:
selector = NodeSelector(graph)
selected = selector.select("Person", name="Tom Hanks")
list(selected)

[(a5b194b:Person {born:1956,name:"Tom Hanks"})]

## Some 'raw' queries

In [10]:
graph.data("MATCH (a:Person)-[]-(m:Movie) RETURN a.name, m.title LIMIT 10")

[{'a.name': 'Emil Eifrem', 'm.title': 'The Matrix'},
 {'a.name': 'Joel Silver', 'm.title': 'The Matrix'},
 {'a.name': 'Lana Wachowski', 'm.title': 'The Matrix'},
 {'a.name': 'Lilly Wachowski', 'm.title': 'The Matrix'},
 {'a.name': 'Hugo Weaving', 'm.title': 'The Matrix'},
 {'a.name': 'Laurence Fishburne', 'm.title': 'The Matrix'},
 {'a.name': 'Carrie-Anne Moss', 'm.title': 'The Matrix'},
 {'a.name': 'Keanu Reeves', 'm.title': 'The Matrix'},
 {'a.name': 'Joel Silver', 'm.title': 'The Matrix Reloaded'},
 {'a.name': 'Lana Wachowski', 'm.title': 'The Matrix Reloaded'}]

## Creation & transactions!

In [28]:
from py2neo import Node, Relationship
from py2neo.database import Schema, Transaction

tx = graph.begin()

andrzej = Node("Person", name="Andrzej Grabowski", born=1952)
kiepscy = Node("Movie", title="Świat wg kiepskich")
pitbull = Node("Movie", title="Pitbull. nowe porządki")
listy = Node("Movie", title="Listy do M. 3")

graph.create(andrzej)
graph.create(kiepscy)
graph.create(pitbull)
graph.create(listy)
graph.create(Relationship(andrzej, "ACTED_IN", kiepscy))
graph.create(Relationship(andrzej, "ACTED_IN", pitbull))
graph.create(Relationship(andrzej, "ACTED_IN", listy))

tx.commit()

## More accurate dates

In [29]:
from py2neo.ext.calendar import GregorianCalendar

calendar = GregorianCalendar(graph)
birth = Relationship(andrzej, "BORN", calendar.date(1952, 3, 15).day)
graph.create(birth)

## Indexing

In [18]:
from py2neo import watch
watch("httpstream")

graph.schema.create_index('Person', 'name')

# graph.schema.drop_index('Person', 'name')

> POST http://localhost:7474/db/data/schema/index/Person [26]
< 200 OK [56]


# ipython-cypher magic

https://github.com/versae/ipython-cypher

In [3]:
%load_ext cypher

In [4]:
%cypher MATCH (a {name: 'Tom Hanks'})-[:ACTED_IN]-(b) RETURN b.title as TomHanksMovies

12 rows affected.


TomHanksMovies
Charlie Wilson's War
The Polar Express
A League of Their Own
Cast Away
Apollo 13
The Green Mile
The Da Vinci Code
Cloud Atlas
That Thing You Do
Joe Versus the Volcano
